## df_shuffle.csv


| Day of Week | Index |
| ----------- | ----- |
| Monday      | 0     |
| Tuesday     | 1     |
| Wednesday   | 2     |
| Thursday    | 3     |
| Friday      | 4     |
| Saturday    | 5     |
| Sunday      | 6     |

In [46]:
sc

In [47]:
import pyproj
import csv
import shapely.geometry as geom
import fiona
import fiona.crs
import shapely
import rtree
import geopandas as gpd
import numpy as np
import operator
# just for display, not for calculation
import pandas as pd
import datetime

In [88]:
# df = pd.read_csv('./df_shuffle.csv')
# df.head(10)

# def wkd(time):
#     return datetime.datetime.strptime(time, '%Y-%m-%d %H:%M:%S').weekday()

# wkd(df.tpep_pickup_datetime[0])
# pd.to_datetime(df.tpep_pickup_datetime).map(lambda x: x.weekday())

# Function

In [73]:
def countLine(partID, records):
    import pyproj
    import csv
    import shapely.geometry as geom
    import fiona
    import fiona.crs
    import shapely
    import rtree
    import geopandas as gpd
    import numpy as np
    import operator
    import pandas as pd
    import datetime
    
    shapefile = '../why_yellow_taxi/Buffer/entr_buffer_100_feet_epsg4269_nad83/entr_buffer_100_feet_epsg4269_nad83.shp'
    entr_buf = gpd.read_file(shapefile)
    entr_buf = entr_buf.to_crs(fiona.crs.from_epsg(2263))
    
    routes = ['Route_' + str(n) for n in range(1, 12)]
    entr2line = []
    for i in xrange(len(entr_buf)):
        lines = []
        for line in list(entr_buf.loc[:,routes].ix[i].dropna().values):
            try:
                line = str(int(line))
            except ValueError:
                pass
            lines.append(line)
        entr2line.append(lines)
    entr_buf['entr2line'] = entr2line
    
    index = rtree.Rtree()
    for idx, geometry in enumerate(entr_buf.geometry):
        index.insert(idx, geometry.bounds)
    

    entr_pair = {}
    pick_entr = {}
    drop_entr = {}
    entr_lines = {}
    
    proj = pyproj.Proj(init='epsg:2263', preserve_units=True)
    
    if partID==0:
        records.next()
    reader = csv.reader(records)
    for row in reader:
        if ((float(row[5])!=0) and float(row[9]!=0)):
            if row[1]:
                wd = datetime.datetime.strptime(row[1], '%Y-%m-%d %H:%M:%S').weekday()
            else:
                wd = None
    
            p = geom.Point(proj(float(row[5]), float(row[6])))
            d = geom.Point(proj(float(row[9]), float(row[10])))
            p_potential = index.intersection((p.x,p.y,p.x,p.y))
            d_potential = index.intersection((d.x,d.y,d.x,d.y))
            p_match = None # The first one match, should be the closest one? No!
            d_match = None
            
            for p_idx in p_potential:
                if entr_buf.geometry[p_idx].contains(p):
                    p_match = p_idx # print 'p',p_idx
                    p_lines = set(entr_buf.entr2line[p_idx])
                    break
            pick_entr[p_match] = pick_entr.get(p_match, 0)+1
            
            for d_idx in d_potential:
                if entr_buf.geometry[d_idx].contains(d):
                    d_match = d_idx # print 'd',d_idx
                    d_lines = set(entr_buf.entr2line[d_idx])
                    break
            drop_entr[d_match] = drop_entr.get(d_match, 0)+1
            
            if ((p_match and d_match) and (p_match != d_match)):
                dirct_lines = tuple(p_lines.intersection(d_lines))
                dirct_lines_wd = (dirct_lines, wd)
                if dirct_lines:
                    #entr_lines[dirct_lines] = entr_lines.get(dirct_lines, 0)+1
                    entr_lines[dirct_lines_wd] = entr_lines.get(dirct_lines_wd, 0)+1
                if p_match > d_match:
                    pair = (d_match, p_match)
                else:
                    pair = (p_match, d_match)
                entr_pair[pair] = entr_pair.get(pair, 0)+1
                
    return entr_lines.items()

In [78]:
def mapper(record):
    for key in record[0][0]:
        yield (key, record[0][1]), record[1]
        
rdd = sc.textFile('./df_shuffle.csv')
counts = rdd.mapPartitionsWithIndex(countLine).flatMap(mapper).reduceByKey(lambda x,y: x+y).collect()  

In [82]:
for i in range(len(counts)):
    if counts[i][0][0] == '6':
        print 'Day of week:{} ; Counts:{}'.format(counts[i][0][1], counts[i][1])

Day of week:5 ; Counts:3
Day of week:3 ; Counts:1
Day of week:0 ; Counts:2
Day of week:2 ; Counts:1


In [90]:
sorted(counts, key=lambda x: x[1], reverse=True)

[(('1', 5), 5),
 (('1', 3), 4),
 ((u'E', 5), 4),
 ((u'C', 5), 4),
 (('6', 5), 3),
 (('1', 0), 3),
 (('1', 4), 3),
 ((u'Q', 0), 3),
 ((u'A', 5), 3),
 ((u'C', 1), 3),
 ((u'R', 0), 3),
 ((u'N', 0), 3),
 (('2', 5), 2),
 (('2', 1), 2),
 ((u'Q', 6), 2),
 ((u'R', 6), 2),
 (('6', 0), 2),
 (('1', 1), 2),
 ((u'F', 4), 2),
 ((u'N', 6), 2),
 (('3', 1), 2),
 (('3', 5), 2),
 ((u'F', 1), 1),
 ((u'C', 6), 1),
 ((u'F', 5), 1),
 ((u'B', 5), 1),
 (('1', 6), 1),
 ((u'E', 6), 1),
 (('6', 3), 1),
 (('3', 4), 1),
 ((u'A', 6), 1),
 ((u'C', 4), 1),
 (('7', 2), 1),
 ((u'A', 4), 1),
 (('5', 0), 1),
 (('2', 3), 1),
 ((u'D', 5), 1),
 ((u'M', 6), 1),
 (('7', 4), 1),
 ((u'C', 2), 1),
 (('3', 3), 1),
 ((u'A', 1), 1),
 (('2', 4), 1),
 ((u'E', 3), 1),
 ((u'L', 4), 1),
 (('6', 2), 1),
 ((u'M', 5), 1),
 ((u'A', 3), 1),
 (('4', 0), 1),
 ((u'L', 0), 1),
 ((u'C', 3), 1),
 ((u'L', 2), 1),
 ((u'E', 1), 1)]